In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from keras.metrics import Precision, Recall

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape and normalize the input data
X_train = X_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
X_test = X_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

# One-hot encode the target labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create a Sequential model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128)

# Evaluate the model
metrics = model.evaluate(X_test, y_test)
print("Test Accuracy:", metrics[1])
print("Precision:", metrics[2])
print("Recall:", metrics[3])

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to file
with open("mnist_model.tflite", "wb") as f:
    f.write(tflite_model)


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
469/469 [==============================] - 242s 500ms/step - loss: 0.1515 - accuracy: 0.9527 - precision: 0.9709 - recall: 0.9402
Epoch 2/10
469/469 [==============================] - 221s 472ms/step - loss: 0.0394 - accuracy: 0.9880 - precision: 0.9891 - recall: 0.9871
Epoch 3/10
469/469 [==============================] - 229s 488ms/step - loss: 0.0257 - accuracy: 0.9920 - precision: 0.9925 - recall: 0.9915
Epoch 4/10
469/469 [==============================] - 224s 478ms/step - loss: 0.0196 - accuracy: 0.9937 - precision: 0.9941 - recall: 0.9934
Epoch 5/10
469/469 [==============================] - 221s 471ms/step - loss: 0.0145 - accuracy: 0.9953 - precision: 0.9955 - recall: 0.9951
Epoch 6/10
469/469 [==============================] - 221s 470ms/step - loss: 0.0107 - accuracy: 0.9964 - precision: 0.9966 - recall: 0.9963
Epoch 7/10
469/469 [==============================] - 221s 471ms/step - loss: 0.0103 - ac

Compression

In [ ]:
!pip install tensorflow_model_optimization
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from tensorflow_model_optimization.sparsity.keras import PolynomialDecay
from tensorflow_model_optimization.sparsity.keras import UpdatePruningStep
from tensorflow_model_optimization.sparsity.keras import PruningSummaries

# Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((-1, 28, 28, 1)).astype('float32') / 255
X_test = X_test.reshape((-1, 28, 28, 1)).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the model architecture
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

# Set up pruning
epochs = 10
batch_size = 128
num_train_samples = X_train.shape[0]
end_step = np.ceil(num_train_samples / batch_size).astype(np.int32) * epochs
pruning_schedule = PolynomialDecay(initial_sparsity=0.10, final_sparsity=0.50, begin_step=0, end_step=end_step)

pruned_model = prune_low_magnitude(model, pruning_schedule=pruning_schedule)

# Compile the pruned model
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for pruning
callbacks = [
    UpdatePruningStep(),
    PruningSummaries(log_dir='/tmp/logs', update_freq='epoch')
]

# Train the pruned model
pruned_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks)

# Strip the pruning wrappers for final model
model_for_export = strip_pruning(pruned_model)

# Convert to TensorFlow Lite with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the compressed model
with open('mnist_model_compressed.tflite', 'wb') as f:
    f.write(tflite_model)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 3.7 MB/s eta 0:00:00
11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
422/422 [==============================] - 208s 481ms/step - loss: 0.1546 - accuracy: 0.9526 - val_loss: 0.0469 - val_accuracy: 0.9872
Epoch 2/10
422/422 [==============================] - 212s 503ms/step - loss: 0.0407 - accuracy: 0.9873 - val_loss: 0.0361 - val_accuracy: 0.9897
Epoch 3/10
422/422 [==============================] - 215s 509ms/step - loss: 0.0271 - accuracy: 0.9914 - val_loss: 0.0407 - val_accuracy: 0.9893
Epoch 4/10
422/422 [==============================] - 214s 506ms/step - loss: 0.0189 - accuracy: 0.9941 - val_loss: 0.0303 - val_accuracy: 0.9922
Epoch 5/10
422/422 [==============================] - 210s 498ms/step - loss: 0.0128 - accuracy: 0.9957 - val_loss: 0.0302 - val_accuracy: 0.9913
Epoch 6/10
422/422 [==============================] - 209s 495ms/step - loss: 0.0101 - accuracy: 0.9966 - val_loss: 0.0272 